In [1]:
import sys
!{sys.executable} -m pip install geocoder
!{sys.executable} -m pip install folium

print('Packages installed.')

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Packages installed.


In [2]:
#!conda install -c conda-forge geopy --yes        # if needed
#!conda install -c conda-forge folium=0.5.0 --yes # if needed

import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files


from geopy.geocoders import Nominatim 
GeoLocator = Nominatim(user_agent='My-IBMNotebook')# convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [3]:
toronto_task2_csv = "toronto_part2.csv"

In [6]:
toronto_neighborhoods = pd.read_csv("toronto_part2.csv")

In [7]:
toronto_neighborhoods.shape
toronto_neighborhoods.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,\nM1ANot assigned\n\n,\nM2ANot assigned\n\n,M9AEtobicoke(Islington Avenue),43.64869,-79.38544
1,\nM1BScarborough(Malvern / Rouge)\n\n,\nM2BNot assigned\n\n,M9BEtobicoke(West Deane Park / Princess Garden...,43.81023,-79.22038
2,\nM1CScarborough(Rouge Hill / Port Union / Hig...,\nM2CNot assigned\n\n,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...,43.78948,-79.17614
3,\nM1EScarborough(Guildwood / Morningside / Wes...,\nM2ENot assigned\n\n,M9ENot assigned,43.76343,-79.17820
4,\nM1GScarborough(Woburn)\n\n,\nM2GNot assigned\n\n,M9GNot assigned,43.76748,-79.22829


In [16]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto Canada are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto Canada are 43.6534817, -79.3839347.


In [17]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_neighborhoods['Latitude'], toronto_neighborhoods['Longitude'], toronto_neighborhoods['Borough'], toronto_neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#87cefa',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)

In [18]:
map_toronto

In [19]:
toronto_data = toronto_neighborhoods[toronto_neighborhoods['Borough'].str.contains("Toronto")].reset_index(drop=True)
print(toronto_data.shape)
toronto_data.head()

(0, 5)


,Postalcode,Borough,Neighborhood,Latitude,Longitude


In [20]:
# I will be using the same coordinates for the previous view
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [21]:
CLIENT_ID = 'LBPMSA0FHGOC522KIRAOT1NFSQDXFFLFU2I12IWGJG00OIUJ' # your Foursquare ID
CLIENT_SECRET = 'V3IG5NPKXEKFXVAUIABOVS0T2IXM2RZCKYLUHWYG2MIXKZBY' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LBPMSA0FHGOC522KIRAOT1NFSQDXFFLFU2I12IWGJG00OIUJ
CLIENT_SECRET:V3IG5NPKXEKFXVAUIABOVS0T2IXM2RZCKYLUHWYG2MIXKZBY


In [22]:
# The following function retrieves the venues given the names and coordinates and stores it into dataframe.
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
toronto_venues = getNearbyVenues(names=toronto_neighborhoods['Neighborhood'],
                                   latitudes=toronto_neighborhoods['Latitude'],
                                   longitudes=toronto_neighborhoods['Longitude']
                                  )

M9AEtobicoke(Islington Avenue)
M9BEtobicoke(West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale)
M9CEtobicoke(Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood)
M9ENot assigned
M9GNot assigned
M9HNot assigned
M9JNot assigned
M9KNot assigned
M9LNorth York(Humber Summit)
M9MNorth York(Humberlea / Emery)
M9NYork(Weston)
M9PEtobicoke(Westmount)
M9REtobicoke(Kingsview Village / St. Phillips / Martin Grove Gardens / Richview Gardens)
M9SNot assigned
M9TNot assigned
M9VEtobicoke(South Steeles / Silverstone / Humbergate / Jamestown / Mount Olive / Beaumond Heights / Thistletown / Albion Gardens)
M9WEtobicokeNorthwest(Clairville / Humberwood / Woodbine Downs / West Humber / Kipling Heights / Rexdale / Elms / Tandridge / Old Rexdale)
M9XNot assigned
M9YNot assigned
M9ZNot assigned


In [24]:
toronto_venues.shape

(271, 7)

In [25]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M9AEtobicoke(Islington Avenue),43.64869,-79.38544,Shangri-La Toronto,43.649129,-79.386557,Hotel
1,M9AEtobicoke(Islington Avenue),43.64869,-79.38544,Four Seasons Centre for the Performing Arts,43.650592,-79.385806,Concert Hall
2,M9AEtobicoke(Islington Avenue),43.64869,-79.38544,Soho House Toronto,43.648734,-79.386541,Speakeasy
3,M9AEtobicoke(Islington Avenue),43.64869,-79.38544,Rosalinda,43.650252,-79.385156,Vegetarian / Vegan Restaurant
4,M9AEtobicoke(Islington Avenue),43.64869,-79.38544,The Keg Steakhouse + Bar - York Street,43.649987,-79.384103,Restaurant


In [30]:
toronto_venues.groupby('Neighborhood').head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M9AEtobicoke(Islington Avenue),43.648690,-79.385440,Shangri-La Toronto,43.649129,-79.386557,Hotel
1,M9AEtobicoke(Islington Avenue),43.648690,-79.385440,Four Seasons Centre for the Performing Arts,43.650592,-79.385806,Concert Hall
2,M9AEtobicoke(Islington Avenue),43.648690,-79.385440,Soho House Toronto,43.648734,-79.386541,Speakeasy
3,M9AEtobicoke(Islington Avenue),43.648690,-79.385440,Rosalinda,43.650252,-79.385156,Vegetarian / Vegan Restaurant
4,M9AEtobicoke(Islington Avenue),43.648690,-79.385440,The Keg Steakhouse + Bar - York Street,43.649987,-79.384103,Restaurant
30,M9BEtobicoke(West Deane Park / Princess Garden...,43.810230,-79.220380,Shoppers Drug Mart,43.809610,-79.222729,Pharmacy
31,M9BEtobicoke(West Deane Park / Princess Garden...,43.810230,-79.220380,A&W,43.809292,-79.223009,Fast Food Restaurant
32,M9BEtobicoke(West Deane Park / Princess Garden...,43.810230,-79.220380,Subway,43.806961,-79.221476,Sandwich Place
33,M9BEtobicoke(West Deane Park / Princess Garden...,43.810230,-79.220380,Pizza Pizza,43.806613,-79.221243,Pizza Place
34,M9BEtobicoke(West Deane Park / Princess Garden...,43.810230,-79.220380,Malvern Arena,43.808594,-79.216634,Skating Rink


In [32]:
toronto_venues.groupby('Venue Category').max()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
ATM,M9ENot assigned,43.763430,-79.178200,HODL Bitcoin ATM - VB Neighborhood,43.762443,-79.183159
Asian Restaurant,M9ZNot assigned,43.786260,-79.280840,One2 Snacks,43.787048,-79.276658
Bakery,M9REtobicoke(Kingsview Village / St. Phillips ...,43.767480,-79.228290,Fabian's Cafe,43.768158,-79.229159
Bank,M9WEtobicokeNorthwest(Clairville / Humberwood ...,43.798899,-79.312939,RBC Royal Bank,43.798236,-79.317952
Bistro,M9MNorth York(Humberlea / Emery),43.726420,-79.232970,Vi Pei Bistro - Bluffs,43.727148,-79.229640
Brazilian Restaurant,M9ZNot assigned,43.648690,-79.385440,Copacabana Grilled Brazilian,43.648333,-79.388151
Bubble Tea Shop,M9BEtobicoke(West Deane Park / Princess Garden...,43.810230,-79.220380,Real Fruit Bubble Tea 真果茶坊,43.806709,-79.222759
Burger Joint,M9WEtobicokeNorthwest(Clairville / Humberwood ...,43.798899,-79.298020,Burger Palace,43.799255,-79.297731
Bus Station,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...,43.789480,-79.176140,Bus stop: 95 Westbound,43.788511,-79.180494


# One Hot encoding the venue Categories

In [33]:
toronto_venue_cat = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_venue_cat

,ATM,Asian Restaurant,Bakery,Bank,Bistro,Brazilian Restaurant,Bubble Tea Shop,Burger Joint,Bus Station,Bus Stop,Butcher,Café,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Concert Hall,Construction & Landscaping,Convenience Store,Dessert Shop,Discount Store,Dog Run,Electronics Store,Fast Food Restaurant,Food Court,Fried Chicken Joint,Gas Station,Gastropub,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hong Kong Restaurant,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Japanese Restaurant,Korean Restaurant,Liquor Store,Lounge,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Motorcycle Shop,Moving Target,Neighborhood,Noodle House,Opera House,Park,Peking Duck Restaurant,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Skating Rink,Smoke Shop,Speakeasy,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Trail,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [34]:
toronto_venue_cat['Neighborhood'] = toronto_venues['Neighborhood'] 

# moving neighborhood column to the first column
fixed_columns = [toronto_venue_cat.columns[-1]] + list(toronto_venue_cat.columns[:-1])
toronto_venue_cat = toronto_venue_cat[fixed_columns]

toronto_venue_cat.head()

,Vietnamese Restaurant,ATM,Asian Restaurant,Bakery,Bank,Bistro,Brazilian Restaurant,Bubble Tea Shop,Burger Joint,Bus Station,Bus Stop,Butcher,Café,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Concert Hall,Construction & Landscaping,Convenience Store,Dessert Shop,Discount Store,Dog Run,Electronics Store,Fast Food Restaurant,Food Court,Fried Chicken Joint,Gas Station,Gastropub,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hong Kong Restaurant,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Japanese Restaurant,Korean Restaurant,Liquor Store,Lounge,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Motorcycle Shop,Moving Target,Neighborhood,Noodle House,Opera House,Park,Peking Duck Restaurant,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Skating Rink,Smoke Shop,Speakeasy,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Trail,Vegetarian / Vegan Restaurant
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,M9AEtobicoke(Islington Avenue),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,M9AEtobicoke(Islington Avenue),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,M9AEtobicoke(Islington Avenue),0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,M9AEtobicoke(Islington Avenue),0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,M9AEtobicoke(Islington Avenue),0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [35]:
toronto_grouped = toronto_venue_cat.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Vietnamese Restaurant,ATM,Asian Restaurant,Bakery,Bank,Bistro,Brazilian Restaurant,Bubble Tea Shop,Burger Joint,Bus Station,Bus Stop,Butcher,Café,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Concert Hall,Construction & Landscaping,Convenience Store,Dessert Shop,Discount Store,Dog Run,Electronics Store,Fast Food Restaurant,Food Court,Fried Chicken Joint,Gas Station,Gastropub,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hong Kong Restaurant,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Japanese Restaurant,Korean Restaurant,Liquor Store,Lounge,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Motorcycle Shop,Moving Target,Noodle House,Opera House,Park,Peking Duck Restaurant,Pharmacy,Pizza Place,Playground,Rental Car Location,Restaurant,Sandwich Place,Seafood Restaurant,Shopping Mall,Skating Rink,Smoke Shop,Speakeasy,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taiwanese Restaurant,Tea Room,Thai Restaurant,Theater,Trail,Vegetarian / Vegan Restaurant
0,M9AEtobicoke(Islington Avenue),0.0,0.000,0.033333,0.00,0.0,0.0,0.033333,0.000000,0.0,0.00,0.0,0.0,0.066667,0.0,0.0,0.0,0.1,0.0,0.066667,0.00,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.033333,0.0,0.0,0.033333,0.000000,0.0,0.033333,0.0,0.033333,0.00,0.0,0.0,0.0,0.033333,0.0,0.0,0.033333,0.033333,0.0,0.0,0.0,0.0,0.0,0.033333,0.00,0.0,0.000000,0.033333,0.0,0.0,0.066667,0.000000,0.0,0.0,0.000000,0.033333,0.033333,0.066667,0.000000,0.00,0.033333,0.0,0.033333,0.033333,0.066667,0.0,0.033333
1,M9BEtobicoke(West Deane Park / Princess Garden...,0.0,0.000,0.000000,0.00,0.0,0.0,0.000000,0.071429,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.071429,0.0,0.0,0.0,0.0,0.214286,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.071429,0.0,0.071429,0.0,0.000000,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.142857,0.142857,0.0,0.0,0.000000,0.071429,0.0,0.0,0.071429,0.000000,0.000000,0.000000,0.071429,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000
2,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...,0.0,0.000,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.0,0.25,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.25,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.250000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000
3,M9ENot assigned,0.0,0.125,0.000000,0.00,0.0,0.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.25,0.125000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.25,0.0,0.000000,0.000000,0.0,0.0,0.125000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0,0.125000,0.000000,0.000000,0.0,0.000000
4,M9GNot assigned,0.0,0.000,0.000000,0.25,0.0,0.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.00,0.0,0.5,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000


In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
import numpy as np

In [38]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M9AEtobicoke(Islington Avenue),Coffee Shop,Restaurant,Theater,Café,Concert Hall,Steakhouse,Gym / Fitness Center,Opera House,Mediterranean Restaurant,Lounge
1,M9BEtobicoke(West Deane Park / Princess Garden...,Fast Food Restaurant,Pharmacy,Pizza Place,Bubble Tea Shop,Grocery Store,Convenience Store,Sandwich Place,Skating Rink,Gym / Fitness Center,Supermarket
2,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...,Bus Station,IT Services,Pharmacy,Fried Chicken Joint,Convenience Store,Dessert Shop,Discount Store,Dog Run,Electronics Store,Fast Food Restaurant
3,M9ENot assigned,Park,Construction & Landscaping,ATM,Tea Room,Convenience Store,Restaurant,Fast Food Restaurant,Dessert Shop,Discount Store,Dog Run
4,M9GNot assigned,Indian Restaurant,Bakery,Supplement Shop,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Dessert Shop,Discount Store,Dog Run,Electronics Store,Fast Food Restaurant



Let's make the model to cluster our Neighborhoods

In [39]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [40]:
# set number of clusters
k_num_clusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=k_num_clusters, random_state=0).fit(toronto_grouped_clustering)
kmeans

KMeans(n_clusters=5, random_state=0)

Checking the labelling of our model

In [41]:
kmeans.labels_[0:100]

array([0, 0, 0, 3, 2, 4, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
      dtype=int32)

In [42]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [44]:
toronto_merged = toronto_neighborhoods

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,\nM1ANot assigned\n\n,\nM2ANot assigned\n\n,M9AEtobicoke(Islington Avenue),43.64869,-79.38544,0,Coffee Shop,Restaurant,Theater,Café,Concert Hall,Steakhouse,Gym / Fitness Center,Opera House,Mediterranean Restaurant,Lounge
1,\nM1BScarborough(Malvern / Rouge)\n\n,\nM2BNot assigned\n\n,M9BEtobicoke(West Deane Park / Princess Garden...,43.81023,-79.22038,0,Fast Food Restaurant,Pharmacy,Pizza Place,Bubble Tea Shop,Grocery Store,Convenience Store,Sandwich Place,Skating Rink,Gym / Fitness Center,Supermarket
2,\nM1CScarborough(Rouge Hill / Port Union / Hig...,\nM2CNot assigned\n\n,M9CEtobicoke(Eringate / Bloordale Gardens / Ol...,43.78948,-79.17614,0,Bus Station,IT Services,Pharmacy,Fried Chicken Joint,Convenience Store,Dessert Shop,Discount Store,Dog Run,Electronics Store,Fast Food Restaurant
3,\nM1EScarborough(Guildwood / Morningside / Wes...,\nM2ENot assigned\n\n,M9ENot assigned,43.76343,-79.17820,3,Park,Construction & Landscaping,ATM,Tea Room,Convenience Store,Restaurant,Fast Food Restaurant,Dessert Shop,Discount Store,Dog Run
4,\nM1GScarborough(Woburn)\n\n,\nM2GNot assigned\n\n,M9GNot assigned,43.76748,-79.22829,2,Indian Restaurant,Bakery,Supplement Shop,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Dessert Shop,Discount Store,Dog Run,Electronics Store,Fast Food Restaurant


In [45]:
toronto_merged_nonan = toronto_merged.dropna(subset=['Cluster Labels'])


Plotting the clusters on the map

In [46]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [47]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k_num_clusters)
ys = [i + x + (i*x)**2 for i in range(k_num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged_nonan['Latitude'], toronto_merged_nonan['Longitude'], toronto_merged_nonan['Neighborhood'], toronto_merged_nonan['Cluster Labels']):
    label = folium.Popup('Cluster ' + str(int(cluster) +1) + '\n' + str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)]
        ).add_to(map_clusters)
        
map_clusters

Cluster 1

In [48]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 0, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,\nM2ANot assigned\n\n,0,Coffee Shop,Restaurant,Theater,Café,Concert Hall,Steakhouse,Gym / Fitness Center,Opera House,Mediterranean Restaurant,Lounge
1,\nM2BNot assigned\n\n,0,Fast Food Restaurant,Pharmacy,Pizza Place,Bubble Tea Shop,Grocery Store,Convenience Store,Sandwich Place,Skating Rink,Gym / Fitness Center,Supermarket
2,\nM2CNot assigned\n\n,0,Bus Station,IT Services,Pharmacy,Fried Chicken Joint,Convenience Store,Dessert Shop,Discount Store,Dog Run,Electronics Store,Fast Food Restaurant
6,\nM2JNorth York(Fairview / Henry Farm / Oriole...,0,Coffee Shop,Supermarket,Gym,Pizza Place,Butcher,Seafood Restaurant,Fried Chicken Joint,Pharmacy,Theater,Gas Station
7,\nM2KNorth York(Bayview Village)\n\n,0,Coffee Shop,Restaurant,Theater,Café,Concert Hall,Steakhouse,Gym / Fitness Center,Opera House,Mediterranean Restaurant,Lounge
8,\nM2LNorth York(York Mills / Silver Hills)\n\n,0,Trail,Dessert Shop,Dog Run,Motorcycle Shop,Bus Stop,Construction & Landscaping,Discount Store,Electronics Store,Fast Food Restaurant,Food Court
10,\nM2NNorth York(Willowdale)South\n\n,0,College Stadium,General Entertainment,Café,Skating Rink,Fast Food Restaurant,Convenience Store,Dessert Shop,Discount Store,Dog Run,Electronics Store
11,\nM2PNorth York(York Mills West)\n\n,0,Gym / Fitness Center,Trail,Playground,Coffee Shop,Intersection,Moving Target,Fast Food Restaurant,Dessert Shop,Discount Store,Dog Run
12,\nM2RNorth York(Willowdale)West\n\n,0,Pizza Place,Vietnamese Restaurant,Intersection,Coffee Shop,Middle Eastern Restaurant,Korean Restaurant,Burger Joint,Gas Station,Bakery,Grocery Store
13,\nM2SNot assigned\n\n,0,Chinese Restaurant,Asian Restaurant,Food Court,Vietnamese Restaurant,Korean Restaurant,Peking Duck Restaurant,Coffee Shop,Rental Car Location,Cantonese Restaurant,Shopping Mall


Cluster 2

In [49]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 1, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,\nM2XNot assigned\n\n,1,Trail,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Convenience Store,Dessert Shop,Discount Store,Dog Run,Electronics Store,Fast Food Restaurant,Food Court


Cluster 3

In [50]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 2, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,\nM2GNot assigned\n\n,2,Indian Restaurant,Bakery,Supplement Shop,Vegetarian / Vegan Restaurant,Fried Chicken Joint,Dessert Shop,Discount Store,Dog Run,Electronics Store,Fast Food Restaurant


Cluster 4

In [51]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 3, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,\nM2ENot assigned\n\n,3,Park,Construction & Landscaping,ATM,Tea Room,Convenience Store,Restaurant,Fast Food Restaurant,Dessert Shop,Discount Store,Dog Run


Cluster 5

In [53]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 4, toronto_merged_nonan.columns[[1] + list(range(5, toronto_merged_nonan.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,\nM2HNorth York(Hillcrest Village)\n\n,4,Ice Cream Shop,Indian Restaurant,Restaurant,Vegetarian / Vegan Restaurant,Food Court,Convenience Store,Dessert Shop,Discount Store,Dog Run,Electronics Store
9,\nM2MNorth York(Willowdale / Newtonbrook)\n\n,4,Ice Cream Shop,Pharmacy,Pizza Place,Coffee Shop,Bistro,Sandwich Place,Liquor Store,Electronics Store,Convenience Store,Dessert Shop


We have successfully cluster Toronto neighbourhood based on venue categories!